### Stage 2: App behaviour is normally distributed

    1) The activity vectors are sampled from a Gaussian distribution
    2) The clusters with normal distributions represent samples generated from a hidden Markov process
    3) To test this, we must first observe whether the data can be split into clusters with Gaussian distributions at a significance level of 0.001
    4) This test will be done via the G-Means algorithm [1]
    
[1] Hamerly, G., & Elkan, C. (n.d.). Learning the k in k -means.


In [ ]:
import numpy as np
import itertools

from scipy import linalg
import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn import mixture

from utils import *
from gmeans import *
import pandas as pd

In [ ]:
time_percentage = 0.8
explained_variance = 0.8
df = pd.read_csv("data/rescuetime_data-ac-min.csv")
data_pd = Clean_DF(df)
data_pd.clean_data(time_percentage=time_percentage)
data_pd.clean_df = data_pd.clean_df.reset_index()
data_pd.get_pca(explained_variance=explained_variance)
data_pd.get_day_time()

## G-Means

#### Algorithm: 

    1) Start from k=1      
    2) Test whether data in each cluster is a Gaussian via the Anderson-Darling test at alpha = 0.0001       
    3) if data is Gaussian, keep center      
    4) else, split cluster into two sub clusters and repeat from 2      


#### Gaussian test:      
    
    1) Run k-means on sub-data X with k=2      
    2) Compute v = c1 - c2 where c1 & c2 are the two new cluster centres found by k-means on X 
    3) Project X onto v Xv = dot(X,v)/dot(v,v)      
    4) Scale Xv to mean 0 and unit variance      
    5) apply Anderson-Darling test to this vector      

In [1]:
import numpy as np
from gmeans import *
A = np.random.normal(loc=5, scale=3, size=(5000,2))
B = np.random.normal(loc=13, scale=2, size=(5000,2))
C = np.random.normal(loc=50, scale=7, size=(5000,2))
D = np.concatenate((A,B,C))

gm = GMeans(verbose=1, trim_points=False)
gm.fit(D)
print(len(gm.cluster_centers_))
gaussian_clusters = gm.cluster_centers_

/Users/asturkmani/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)




-------------------------------------
iteration:  1
Set of gaussian clusters:  []
Set of temporary clusters:  [array([ 50.14614257,  49.97228449]), array([ 9.03064532,  9.05167045])]


-------------------------------------
iteration:  2
Set of gaussian clusters:  [array([ 50.14614257,  49.97228449])]
Set of temporary clusters:  [array([ 12.90954   ,  12.97854563]), array([ 4.92419094,  4.89442074])]


-------------------------------------
iteration:  3
Set of gaussian clusters:  [array([ 50.14614257,  49.97228449]), array([ 50.14614257,  49.97228449])]
Set of temporary clusters:  [array([ 12.90954   ,  12.97854563]), array([ 4.92419094,  4.89442074])]


-------------------------------------
iteration:  4
Set of gaussian clusters:  [array([ 50.14614257,  49.97228449]), array([ 50.14614257,  49.97228449]), array([ 50.14614257,  49.97228449])]
Set of temporary clusters:  [array([ 12.90954   ,  12.97854563]), array([ 4.92419094,  4.89442074])]


-------------------------------------
iter

18


In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

kmeans = KMeans(n_clusters=len(gaussian_clusters), init=gaussian_clusters)
kmeans.fit(D)
c = kmeans.labels_
trace = go.Scatter(x=D[:,0], y=D[:,1], mode='markers', marker=dict(
        size='10',
        color = c, #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    ))
data= [trace]
py.iplot(data, filename='K-Means inerta')

In [ ]:
gm = GMeans()
gm.fit(data_pd.activity_vector)
print(len(gm.cluster_centers_))

In [ ]:
c = gm.labels_
x = data_pd.pca_data[:,0]
y = data_pd.pca_data[:,1]
z = data_pd.pca_data[:,2]
t = data_pd.clean_df['Activity']
t = data_pd.clean_df['Activity'].tolist()
t = ['-'.join(x) for x in t]
for i in range(0,len(t)):
    t[i] = str(gm.labels_[i]) + '---' + t[i]
    
trace1 = go.Scatter3d(x=x,y=y,z=z,text=t,mode='markers',marker=dict(size=12,color=c, colorscale='Viridis',opacity=0.8))
data = [trace1]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='PCA-3 Visualization with G-Means')

In [ ]:
from sklearn.manifold import TSNE
tsne_pca = TSNE(n_components=3, verbose=0, perplexity=30, n_iter=5000)
tsne_results_pca = tsne_pca.fit_transform(data_pd.pca_data)

In [ ]:
c = gm.labels_
x = tsne_results_pca[:,0]
y = tsne_results_pca[:,1]
z = tsne_results_pca[:,2]
# t = data_pd.clean_df['Activity']
# t = data_pd.clean_df['Activity'].tolist()
# t = ['-'.join(x) for x in t]

trace1 = go.Scatter3d(x=x,y=y,z=z,text=t,mode='markers',marker=dict(size=12,color=c, colorscale='RdYlGn',opacity=0.8))
data = [trace1]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='t-SNE PCA 90% variance Visualization')

In [ ]:
lowest_bic = np.infty
bic = []
n_components_range = range(1, 20)
cv_types = ['spherical', 'tied', 'diag', 'full']
X = data_pd.pca_data
for cv_type in cv_types:
    for n_components in n_components_range:
        # Fit a Gaussian mixture with EM
        gmm = mixture.GaussianMixture(n_components=n_components,
                                      covariance_type=cv_type)
        gmm.fit(X)
        bic.append(gmm.bic(X))
        if (bic[-1] < lowest_bic):
            lowest_bic = bic[-1]
            best_gmm = gmm

bic = np.array(bic)
color_iter = itertools.cycle(['navy', 'turquoise', 'cornflowerblue',
                              'darkorange'])
clf = best_gmm
bars = []
lowest_bic = max(bic)

In [ ]:
print(bic)
# Plot the BIC scores
spl = plt.subplot(2, 1, 1)
for i, (cv_type, color) in enumerate(zip(cv_types, color_iter)):
    xpos = np.array(n_components_range) + .2 * (i - 2)
    bars.append(plt.bar(xpos, bic[i * len(n_components_range):
                                  (i + 1) * len(n_components_range)],
                        width=.2, color=color))
plt.xticks(n_components_range)
plt.ylim([bic.min() * 1.01 - .01 * bic.max(), bic.max()])
plt.title('BIC score per model')
xpos = np.mod(bic.argmin(), len(n_components_range)) + .65 +\
    .2 * np.floor(bic.argmin() / len(n_components_range))
plt.text(xpos, bic.min() * 0.97 + .03 * bic.max(), '*', fontsize=14)
spl.set_xlabel('Number of components')
spl.legend([b[0] for b in bars], cv_types)

plt.show()

In [ ]:
# A = np.random.normal(loc=5,scale=5,size=(500,2))
# B = np.random.normal(loc=50,scale=5,size=(500,2))

# D = np.concatenate((A,B))
# from sklearn.decomposition import PCA

# kmeans = KMeans(n_clusters=1)
# kmeans.fit(D)
# clusters = kmeans.cluster_centers_
# print(clusters)   

# pca = PCA(n_components=1)
# pca_data = pca.fit(D)
# m = pca_data.components_ * np.sqrt(2*pca_data.explained_variance_/np.pi)

# print("new initial clusters: ", clusters+m, clusters-m)

# ddd = np.zeros((2,2))
# ddd[0,:] = clusters+m
# ddd[1,:] = clusters-m

# kmeans = KMeans(n_clusters=2, init = ddd)
# kmeans.fit(D)
# clusters = kmeans.cluster_centers_
# print(clusters)